In [1]:
MODEL_FLAGS="--attention_resolutions 32,16,8 --class_cond False --diffusion_steps 1000 --image_size 128 --learn_sigma True --noise_schedule cosine --num_channels 128 --num_heads 4 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm True"
MODEL_PATH="--model_path /home/minhnh/project_drive/CV/seismic-diffusion-models/model200000.pt"
SAMPLE_FLAGS="--mask_ratio 0.4 --noise_scale 0.1 --gradient_scale 0.5"
!python3 sample.py $MODEL_FLAGS $MODEL_PATH $SAMPLE_FLAGS --cuda_devices '7' --batch_size 150 --mode test --data_dir '/home/minhnh/project_drive/CV/seismic-dataset' --dataset 'F3'

Traceback (most recent call last):
  File "sample.py", line 22, in <module>
    from piq import ssim, psnr
ModuleNotFoundError: No module named 'piq'
